In [226]:
%load_ext autoreload
%autoreload 2
import predictions_new as P
import ipywidgets as W
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
from importlib import reload
from matplotlib import pyplot as plt
import traceback
from IPython.display import display, HTML, clear_output
display(HTML("<style>.container{width: 100%}</style>"))
P = reload(P)
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [206]:
conf = json.load(open("./ids.json", "r"))

selected_materials = conf["selected_materials"]
selected_plants = conf["selected_plants"]

selected_plants_dict = {v: i for i, v in enumerate(selected_plants)}

In [207]:
data = P.read_preproc_cut_transcations_data(
    path="./.data/tr.parquet", 
    plants=selected_plants,
    materials=selected_materials,
)
imp = pd.read_csv("./.data/imp.csv")
sat = pd.read_csv("./.data/sat.csv")
imp = imp[imp["material"].isin(selected_materials)].reset_index()[["material", "g_i_c", "plant"]]
desc = data.describe()
desc

,client_id,sales_count,sales_sum,product_id,shop_id
count,330427.000000,330427.000000,330427.000000,330427.000000,330427.000000
mean,5736.605138,1.358387,130.058386,42.813227,4.672221
std,3251.402627,1.665964,171.650332,28.611921,2.974433
min,0.000000,0.022000,0.210000,0.000000,0.000000
25%,2865.000000,0.836000,44.400000,17.000000,2.000000
50%,5829.000000,1.000000,77.790000,39.000000,5.000000
75%,8503.000000,1.528000,159.990000,67.000000,8.000000
max,11382.000000,591.500000,32520.670000,99.000000,9.000000


In [244]:
P = reload(P)
DPI = 120
def show_predictions(shop_id, product_id, today, predict_days):
    today = pd.to_datetime(today)
    print("Loading...")
    
    fig, ax = plt.subplots(2, 2, figsize=(8, 4), dpi=DPI)
    plt.margins(0, 0)
    
    ax_demand = ax[0][0]
    demand, prev_demand = P.predict_demand(data, product_id=product_id, shop_id=shop_id, current_date=today, prediction_length=predict_days)
    x = list(range(len(demand)))
    prev_demand = prev_demand[-20:]
    x_pdemand = list(range(-len(prev_demand), 0))
    ax_demand.bar(x, demand, label="prediction")
    ax_demand.bar(x_pdemand, prev_demand, label="real data")
    ax_demand.set_title(f"Demand prediction")
    ax_demand.set_xlabel("Future days")
    ax_demand.set_ylim(0, max(max(demand), max(prev_demand)))
    
    ax_stock = ax[0][1]
    stock, prev_stock= P.predict_stock_quantity(data, product_id=product_id, shop_id=shop_id, current_date=today, prediction_length=predict_days)
    prev_stock = prev_stock[-20:]
    x_pstock = list(range(-len(prev_stock), 0))
    ax_stock.bar(x_pstock, prev_stock, label="real data")
    ax_stock.bar(x, stock, label="prediction")
    ax_stock.set_title(f"Stock quantity prediction")
    ax_stock.set_xlabel("Future days")
    ax_stock.set_ylim(0, max(max(stock), max(prev_stock)))
    
    ax_sat = ax[1][0]
    sub_sat = sat[sat["plant"]==selected_plants[shop_id]]
    sub_sat = sub_sat[pd.to_datetime(sub_sat["chq_date"])==today]
    y, x = np.histogram(sub_sat["satisfaction"])
    x = np.convolve(x, np.ones((2,))/2, mode='valid')
    ax_sat.bar(x, y, (x[1] - x[0])*0.9)
    ax_sat.set_title("Daily mean customer satisfaction")
    ax_sat.set_ylabel("Amount of customers")
    ax_sat.set_xlabel("Satisfaction score")
    
    ax_imp = ax[1][1]
    loc_imp_ = imp[imp["plant"]==selected_plants[shop_id]].reset_index()
    loc_imp_ = loc_imp_[["material", "g_i_c"]].rename(columns={"g_i_c": "General Importance Score"})
    loc_imp = loc_imp_.head()
    ax_imp.set_title("Top 5 most crucial goods")
    ax_imp.table(cellText=loc_imp.values, colLabels=loc_imp.columns, loc='best')
    loc_imp = loc_imp_[loc_imp_["material"] == selected_materials[product_id]]
    loc_imp = loc_imp.rename(columns={"material": "Selected Material"})
    ax_imp.table(cellText=loc_imp.values, colLabels=loc_imp.columns, loc='bottom')
    ax_imp.axis('off')
    ax_imp.axis('tight')
    ax_imp.set_ylim(0.2, 0.8)
    
    fig.tight_layout()
    clear_output()
    print("Done.")
    return None

In [248]:
W.interactive(
    show_predictions,
    shop_id = W.Dropdown(options=list(range(int(desc["shop_id"]["max"]) + 1)), value=2),
    product_id = W.IntSlider(min=0, max=int(desc["product_id"]["max"]), value=24),
    predict_days=W.IntSlider(min=3, max=15, value=14),
    today=W.DatePicker(value=data["chq_date"].max() - timedelta(days=5))
)

interactive(children=(Dropdown(description='shop_id', index=2, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=2…